In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from Corrfunc.mocks.DDtheta_mocks import DDtheta_mocks
from scipy.optimize import curve_fit

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from Corrfunc.mocks.DDtheta_mocks import DDtheta_mocks
from scipy.optimize import curve_fit

# =========================================================
# 1. Load Data and Define Functions
# =========================================================
def load_subset_data(filename):
    """Load RA/DEC from FITS file."""
    with fits.open(filename) as hdulist:
        data = hdulist[1].data
        ra = data['RA']
        dec = data['DEC']
    return ra, dec

def compute_acf(ra_data_rad, dec_data_rad, ra_rand_rad, dec_rand_rad, theta_bins, n_data, n_rand):
    """Compute ACF for a given dataset."""
    # Pair counts
    DD = DDtheta_mocks(
        autocorr=1, nthreads=4, binfile=theta_bins,
        RA1=ra_data_rad, DEC1=dec_data_rad,
        RA2=ra_data_rad, DEC2=dec_data_rad
    )
    DR = DDtheta_mocks(
        autocorr=0, nthreads=4, binfile=theta_bins,
        RA1=ra_data_rad, DEC1=dec_data_rad,
        RA2=ra_rand_rad, DEC2=dec_rand_rad
    )
    RR = DDtheta_mocks(
        autocorr=1, nthreads=4, binfile=theta_bins,
        RA1=ra_rand_rad, DEC1=dec_rand_rad,
        RA2=ra_rand_rad, DEC2=dec_rand_rad
    )
    
    # Normalize pair counts
    dd, dr, rr = DD['npairs'], DR['npairs'], RR['npairs']
    dd_norm = n_data * (n_data - 1) / 2
    dr_norm = n_data * n_rand
    rr_norm = n_rand * (n_rand - 1) / 2
    dd_normalized = dd / dd_norm
    dr_normalized = dr / dr_norm
    rr_normalized = rr / rr_norm
    
    # Compute w(theta)
    with np.errstate(divide='ignore', invalid='ignore'):
        w_theta = (dd_normalized - 2 * dr_normalized + rr_normalized) / rr_normalized
    
    return w_theta

In [2]:
# =========================================================
# 2. Bootstrap Error Estimation
# =========================================================
def bootstrap_errors(ra_data, dec_data, ra_rand, dec_rand, theta_bins, n_bootstrap=100):
    """Compute bootstrap errors for w(theta)."""
    n_data = len(ra_data)
    n_rand = len(ra_rand)
    ra_rand_rad = np.radians(ra_rand)
    dec_rand_rad = np.radians(dec_rand)
    bootstrap_w = []
    
    for _ in range(n_bootstrap):
        # Resample data with replacement
        indices = np.random.choice(n_data, n_data, replace=True)
        ra_boot = ra_data[indices]
        dec_boot = dec_data[indices]
        ra_boot_rad = np.radians(ra_boot)
        dec_boot_rad = np.radians(dec_boot)
        
        # Compute ACF for this bootstrap sample
        w_boot = compute_acf(ra_boot_rad, dec_boot_rad, ra_rand_rad, dec_rand_rad, theta_bins, n_data, n_rand)
        bootstrap_w.append(w_boot)
    
    # Compute mean and standard deviation
    bootstrap_w = np.array(bootstrap_w)
    w_mean = np.nanmean(bootstrap_w, axis=0)
    w_std = np.nanstd(bootstrap_w, axis=0, ddof=1)
    return w_mean, w_std

In [3]:
# =========================================================
# 3. Power-Law Fit with Errors
# =========================================================
def power_law(theta, A, delta):
    return A * theta**(-delta)

In [6]:
# =========================================================
# 4. Main Workflow
# =========================================================
# Load data
ra_data, dec_data = load_subset_data('north_subset.fits')

# Generate random catalog (same as before)
n_data = len(ra_data)
n_rand = 50 * n_data
ra_min, ra_max = np.min(ra_data), np.max(ra_data)
dec_min, dec_max = np.min(dec_data), np.max(dec_data)
ra_rand = np.random.uniform(ra_min, ra_max, n_rand)
dec_rand = np.random.uniform(dec_min, dec_max, n_rand)

# Angular bins
theta_bins = np.logspace(-1., 2, 100)
theta_centers = (theta_bins[1:] + theta_bins[:-1]) / 2

# Compute original ACF
ra_data_rad = np.radians(ra_data)
dec_data_rad = np.radians(dec_data)
ra_rand_rad = np.radians(ra_rand)
dec_rand_rad = np.radians(dec_rand)
w_theta = compute_acf(ra_data_rad, dec_data_rad, ra_rand_rad, dec_rand_rad, theta_bins, n_data, n_rand)

# Mask invalid bins
valid_mask = (w_theta > -1.0) & np.isfinite(w_theta)  # Exclude extreme outliers
theta_valid = theta_centers[valid_mask]
w_valid = w_theta[valid_mask]

# Compute bootstrap errors
n_bootstrap = 100  # Adjust based on computational resources
w_mean, w_std = bootstrap_errors(ra_data, dec_data, ra_rand, dec_rand, theta_bins, n_bootstrap)
w_valid_mean = w_mean[valid_mask]
w_valid_std = w_std[valid_mask]

# Fit power law with bootstrap errors
popt, pcov = curve_fit(power_law, theta_valid, w_valid_mean, sigma=w_valid_std, p0=[1.0, 0.7])
A_fit, delta_fit = popt
A_err, delta_err = np.sqrt(np.diag(pcov))

# Bootstrap errors for parameters
params_bootstrap = []
for i in range(n_bootstrap):
    # Resample data with replacement
    indices = np.random.choice(n_data, n_data, replace=True)
    ra_boot = ra_data[indices]
    dec_boot = dec_data[indices]
    ra_boot_rad = np.radians(ra_boot)
    dec_boot_rad = np.radians(dec_boot)
    
    # Compute ACF for this bootstrap sample
    w_b = compute_acf(ra_boot_rad, dec_boot_rad, ra_rand_rad, dec_rand_rad, theta_bins, n_data, n_rand)
    
    # Apply same validity mask and fit
    try:
        popt_b, _ = curve_fit(power_law, theta_valid, w_b[valid_mask], p0=[1.0, 0.7])
        params_bootstrap.append(popt_b)
    except:
        pass
params_bootstrap = np.array(params_bootstrap)
A_std_bs = np.std(params_bootstrap[:, 0], ddof=1)
delta_std_bs = np.std(params_bootstrap[:, 1], ddof=1)

print(f"Fit (Covariance Errors): A = {A_fit:.3f} ± {A_err:.3f}, δ = {delta_fit:.3f} ± {delta_err:.3f}")
print(f"Fit (Bootstrap Errors): A = {A_fit:.3f} ± {A_std_bs:.3f}, δ = {delta_fit:.3f} ± {delta_std_bs:.3f}")

/tmp/ipykernel_29392/1043120734.py:26: RuntimeWarning: Mean of empty slice
  w_mean = np.nanmean(bootstrap_w, axis=0)
/home/k3vinpaul/miniconda3/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/k3vinpaul/miniconda3/lib/python3.12/site-packages/scipy/optimize/_minpack_py.py:982: RuntimeWarning: divide by zero encountered in divide
  transform = 1.0 / sigma
/tmp/ipykernel_29392/1092815451.py:38: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(power_law, theta_valid, w_valid_mean, sigma=w_valid_std, p0=[1.0, 0.7])


Fit (Covariance Errors): A = 1.000 ± inf, δ = 0.700 ± inf
Fit (Bootstrap Errors): A = 1.000 ± 0.195, δ = 0.700 ± 0.187
